In [ ]:
using ProgressBars;

using LinearAlgebra, Plots, FFTW;

import LinearSolve as LS
using SparseArrays


function central_diff_vec(vec, h):
    vec_ip1 = circshift(vec,-1)
    vec_im1 = circshift(vec,1)
    return (vec_ip1-vec_im1)/(2*h)
end

function create_mathcalF(x, U_n, PDL_matrix, M_n, Δt, h, ν)
    F_hM_n = F_func(M_n, h)
    δ_hx = central_diff_vec(x)
    Hδ_hx=H_func(δ_hx)
    return x-U_n+Δt*(ν*PDL_matrix*x+ Hδ_hx+F_hM_n)
end

function create_Jacobian(N_h, U, PDL_matrix, Δt, h, ν)
    J_F = zeros(N_h, N_h)

    PDL_subdiag = [PDL_matrix[i+1,i] for i in 1:n_H-1]   # subdiagonal
    PDL_diag  = [PDL_matrix[i,i]   for i in 1:n_H]     # main diagonal
    PDL_supdiag = [PDL_matrix[i,i+1] for i in 1:n_H-1]   # superdiagonal

    δ_hU = central_diff_vec(U,h)

    J_F += I + Δt*ν*Tridiagonal(PDL_subdiag, PDL_diag, PDL_supdiag)+Tridiagonal(-δ_hU,0,δ_hU)
    return J_F
end 

function new_HJB_step(num_iter_HJB, N_h, U_n, PDL_matrix, Δt, h, ν)
    U_np1 = U_n
    for _ in num_iter_HJB
        J_F = create_Jacobian(N_h, U_np1, PDL_matrix, Δt, h, ν)
        mathcalFU_np1 = create_mathcalF(U_np1, U_n, PDL_matrix, M_n, Δt, h, ν)
        δ_k =  J_F \ mathcalFU_np1
        U_np1 = U_np1 - δ_k
    end
    return U_np1
end

function new_HJB_solve(N_h, M_mat, PDL_matrix, Δt, h, ν)
    U_mat = Array{Float64}(undef, N_h, N_T)
    U_mat[:,1] = G_h(M_mat[:,1])
end

function new_FPK_step(U_np1, M_np1, DPL_matrix, Δt, h, ν)
       
    subdiag_temp = -1*(circshift(U_np1,-1)-circshift(U_np1,1))
    supdiag_temp = (circshift(U_np1,-2)-U_np1)
    print("size1: ", size(subdiag_temp))
    subdiag = subdiag_temp[1:end-1]
    supdiag = supdiag_temp[1:end-1]
    print("size2: ", size(subdiag))

    T = Tridiagonal(subdiag, zeros(size(U_np1)[1]), supdiag)
    new_T = sparse(T)
    new_T[1, end] = -(U_np1[1]-U_np1[end-2])
    new_T[end, 1] = U_np1[2]-U_np1[end]

    prob = LS.LinearProblem(I +Δt*(ν*DPL_matrix+1/(2*h^2)*new_T), M_np1)
    sol = LS.solve(prob)

    M_n = sol.u
    return M_n

end

function new_FPK_solve(U_mat, M_T, N_h, Δt, N_T, h, ν, α, R, C_H)
    M_mat = Array{Float64}(undef, N_h, N_T)
    M_mat[:,end] = M_T
    DPL_mat = create_DPL_matrix(N_h, α, h, R)
    for j in 1:(N_T-1)
        M_mat[:,N_T-j] = new_FPK_step(U_mat[:, N_T-j+1], M_mat[:, N_T-j+1], DPL_mat, Δt, h, ν)
    end
    return M_mat
end


function MFG_solve(M_T, cv)
    (α, h, N_h, Δt, N_T, ν, num_it_MFG, num_it_HJB, x_vec, R, δ, C_H) = cv;
    M_mat = Array{Float64}(undef, N_h, N_T); M_mat .= M_T;
    U_mat = Array{Float64}(undef, N_h, N_T);
    for _ in ProgressBar(1:num_it_MFG)
        U_mat = HJB_solve(M_mat, num_it_HJB, x_vec, N_h, Δt, N_T, h, ν, α, R, δ, C_H)
        M_mat = new_FPK_solve(U_mat, M_T, N_h, Δt, N_T, h, ν, α, R, C_H)
    end
    return (U_mat, M_mat)
end



#M_T

m_T_func_unnorm = (x) -> exp(-(x-0.5)^2/0.1^2)
oneOverC, error = quadgk(m_T_func_unnorm, 0, 1);

function m_T_func(x)
    return 1/oneOverC * m_T_func_unnorm(x) 
end